In [1]:
import gc
import os
import cv2
import sys
import json
import time
import timm
import tqdm
import torch
import random
import sklearn.metrics

from PIL import Image
from pathlib import Path
from functools import partial
from contextlib import contextmanager

import numpy as np
import scipy as sp
import pandas as pd
import torch.nn as nn

from torch.optim import Adam, SGD, AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader, Dataset
from albumentations import Compose, Normalize, Resize
from albumentations.pytorch import ToTensorV2

os.environ["CUDA_VISIBLE_DEVICES"]="0"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pd.set_option('display.max_columns', None)
device

device(type='cuda')

In [2]:
!nvidia-smi

Thu Apr  7 22:22:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:04:00.0 Off |                  N/A |
| 45%   85C    P2   183W / 215W |   5240MiB /  7982MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:09:00.0 Off |                  N/A |
| 80%   

In [3]:
train_metadata = pd.read_csv("../iNaturalist2018/iNaturalist2018-Plantae-train.csv")
test_metadata = pd.read_csv("../iNaturalist2018/iNaturalist2018-Plantae-test.csv")
val_metadata = pd.read_csv("../iNaturalist2018/iNaturalist2018-Plantae-val.csv")

In [4]:
train_metadata['image_path'] = train_metadata['file_name'].apply(lambda x: os.path.join('/Data-10T/iNaturalist2018/', x))
test_metadata['image_path'] = test_metadata['file_name'].apply(lambda x: os.path.join('/Data-10T/iNaturalist2018/', x))
val_metadata['image_path'] = val_metadata['file_name'].apply(lambda x: os.path.join('/Data-10T/iNaturalist2018/', x))

In [5]:
tmp = train_metadata.drop_duplicates(subset='class_id')

In [6]:
genus_2_class_indexes = {}
for genus_id in tqdm.tqdm(train_metadata['genus_id'].unique(), total = len(train_metadata['genus_id'].unique())):
    genus_2_class_indexes[genus_id] = sorted(train_metadata[train_metadata['genus_id'] == genus_id].class_id.unique())
    
    
family_2_class_indexes = {}
for family_id in tqdm.tqdm(train_metadata['family_id'].unique(), total = len(train_metadata['family_id'].unique())):
    family_2_class_indexes[family_id] = sorted(train_metadata[train_metadata['family_id'] == family_id].class_id.unique())

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 246/246 [00:00<00:00, 2038.36it/s]


In [7]:
@contextmanager
def timer(name):
    t0 = time.time()
    LOGGER.info(f'[{name}] start')
    yield
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s.')

    
def init_logger(log_file='train.log'):
    from logging import getLogger, DEBUG, FileHandler,  Formatter,  StreamHandler
    
    log_format = '%(asctime)s %(levelname)s %(message)s'
    
    stream_handler = StreamHandler()
    stream_handler.setLevel(DEBUG)
    stream_handler.setFormatter(Formatter(log_format))
    
    file_handler = FileHandler(log_file)
    file_handler.setFormatter(Formatter(log_format))
    
    logger = getLogger('Herbarium')
    logger.setLevel(DEBUG)
    logger.addHandler(stream_handler)
    logger.addHandler(file_handler)
    
    return logger

LOG_FILE = 'iNat18-ViT_base_patch32_224-TaxFormer.log'
LOGGER = init_logger(LOG_FILE)


def seed_torch(seed=777):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

SEED = 777
seed_torch(SEED)

In [8]:
N_CLASSES = len(train_metadata['class_id'].unique())
N_GENERA = len(train_metadata['genus_id'].unique())
N_FAMILY = len(train_metadata['family_id'].unique())


class TrainDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_path = self.df['image_path'].values[idx]
        species_label = self.df['class_id'].values[idx]
        genus_label = self.df['genus_id'].values[idx]
        family_label = self.df['family_id'].values[idx]

        image = cv2.imread(file_path)
        
        try:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        except:
            print(file_path)

        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        
        return image, species_label, genus_label, family_label

In [9]:
MODEL_NAME = 'tax_former'
NUM_CLASSES = [N_CLASSES, N_GENERA, N_FAMILY]

model = timm.create_model(MODEL_NAME,  num_classes= NUM_CLASSES, pretrained=False)

In [10]:
[N_CLASSES, N_GENERA, N_FAMILY]

[2917, 1312, 246]

In [11]:
model_mean = list(model.default_cfg['mean'])
model_std = list(model.default_cfg['std'])

In [12]:
# model

In [13]:
WIDTH, HEIGHT = 224, 224

from albumentations import Cutout, RandomGridShuffle, MultiplicativeNoise, HueSaturationValue, RandomCrop, HorizontalFlip, VerticalFlip, RandomBrightnessContrast, CenterCrop, PadIfNeeded, RandomResizedCrop, ShiftScaleRotate, Blur, JpegCompression, RandomShadow

def get_transforms(*, data):
    assert data in ('train', 'valid')

    if data == 'train':
        return Compose([
            Resize(WIDTH, HEIGHT),
            #HorizontalFlip(p=0.5),
            #VerticalFlip(p=0.5),
            #ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.25, rotate_limit=45, p=.75),
            #JpegCompression(p=0.25, quality_lower=50, quality_upper=100),
            #Blur(blur_limit=(7, 7), p=0.1),
            #RandomGridShuffle(grid=(3, 3), p=0.1),
            #RandomBrightnessContrast(p=0.3),
            #HueSaturationValue(p=0.2),
            #MultiplicativeNoise(multiplier=[0.8, 1.2], elementwise=True, p=0.1),
            #Cutout(num_holes=15, max_h_size=20, max_w_size=20, fill_value=128, p=1),
            Normalize(
                mean=model_mean,
                std=model_std,
            ),
            ToTensorV2(),
        ])

    elif data == 'valid':
        return Compose([
            #PadIfNeeded(WIDTH, HEIGHT),
            Resize(WIDTH, HEIGHT),
            Normalize(mean=model_mean, std=model_std),
            ToTensorV2(),
        ])

In [14]:
# Adjust BATCH_SIZE and ACCUMULATION_STEPS to values that if multiplied results in 64 !!!!!1
BATCH_SIZE = 16
ACCUMULATION_STEPS = 4
EPOCHS = 100
WORKERS = 8

train_dataset = TrainDataset(train_metadata, transform=get_transforms(data='train'))
valid_dataset = TrainDataset(val_metadata, transform=get_transforms(data='valid'))
test_dataset = TrainDataset(test_metadata, transform=get_transforms(data='valid'))

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=WORKERS)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=WORKERS)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=WORKERS)

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import f1_score, accuracy_score
import tqdm
from timm.scheduler import CosineLRScheduler


with timer('Train model'):
    accumulation_steps = ACCUMULATION_STEPS
    n_epochs = EPOCHS
    num_steps_per_epoch = len(train_loader)
    lr = 0.01
    
    model.to(device)
    
    optimizer = SGD(model.parameters(), lr=lr, momentum=0.9)
    scheduler = CosineLRScheduler(optimizer, t_initial=20, lr_min=0.0001, cycle_decay = 0.85, cycle_limit = 5)
    
    criterion = nn.CrossEntropyLoss()
    best_score = 0.
    best_loss = np.inf
        
    lrs = []
    
    for epoch in range(n_epochs):
        
        num_updates = epoch * num_steps_per_epoch
        
        start_time = time.time()

        model.train()
        avg_loss = 0.

        optimizer.zero_grad()

        for i, (images, species_labels, genus_labels, family_labels) in tqdm.tqdm(enumerate(train_loader)):
            
            images = images.to(device)            
            species_labels = species_labels.to(device)
            genus_labels = genus_labels.to(device)
            family_labels = family_labels.to(device)
            
            species_preds, genus_preds, family_preds = model(images)

            loss_species = criterion(species_preds, species_labels)            
            loss_genus   = criterion(genus_preds, genus_labels) 
            loss_family  = criterion(family_preds, family_labels) 
            loss = (loss_species + loss_genus + loss_family) / 3   
            
            # Scale the loss to the mean of the accumulated batch size
            avg_loss += loss.item() / len(train_loader)
            loss = loss / accumulation_steps
            loss.backward()
            if (i - 1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()
            
            # TODO: FIX must be inside if that is above!!
            num_updates += 1
            scheduler.step_update(num_updates=num_updates)
            

        model.eval()
        avg_val_loss = 0.
        preds = np.zeros((len(valid_dataset)))
        preds_raw = []

        for i, (images, species_labels, genus_labels, family_labels) in enumerate(valid_loader):

            images = images.to(device)            
            species_labels = species_labels.to(device)
            genus_labels = genus_labels.to(device)
            family_labels = family_labels.to(device)
            
            with torch.no_grad():
                species_preds, genus_preds, family_preds = model(images)
            
            preds[i * BATCH_SIZE: (i+1) * BATCH_SIZE] = species_preds.argmax(1).to('cpu').numpy()
            preds_raw.extend(species_preds.to('cpu').numpy())

            
            loss_species = criterion(species_preds, species_labels)            
            loss_genus   = criterion(genus_preds, genus_labels) 
            loss_family  = criterion(family_preds, family_labels) 
            loss = (loss_species + loss_genus + loss_family) / 3    
                    
            avg_val_loss += loss.item() / len(valid_loader)
        
        lrs.append(optimizer.param_groups[0]["lr"])
        scheduler.step(epoch + 1)
        
        score = f1_score(val_metadata['class_id'], preds, average='macro')
        accuracy = accuracy_score(val_metadata['class_id'], preds)

        elapsed = time.time() - start_time

        LOGGER.debug(f'  Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f} F1: {score:.4f}  Acc: {accuracy:.4f} LR: {optimizer.param_groups[0]["lr"]:.4f}time: {elapsed:.0f}s')
        
        if accuracy>best_score:
            best_score = accuracy
            LOGGER.debug(f'  Epoch {epoch+1} - Save Best Accuracy: {best_score:.6f} Model')
            torch.save(model.state_dict(), f'iNat18-ViT_base_patch32_224-TaxFormer-best_accuracy.pth')

        if avg_val_loss<best_loss:
            best_loss = avg_val_loss
            LOGGER.debug(f'  Epoch {epoch+1} - Save Best Loss: {best_loss:.4f} Model')
            torch.save(model.state_dict(), f'iNat18-ViT_base_patch32_224-TaxFormer-best_loss.pth')

2022-04-07 22:22:51,990 INFO [Train model] start
1348it [02:54,  7.68it/s]Corrupt JPEG data: 94 extraneous bytes before marker 0xd9
4867it [10:39,  7.71it/s]Corrupt JPEG data: premature end of data segment
7054it [15:26,  7.62it/s]
2022-04-07 22:38:37,458 DEBUG   Epoch 1 - avg_train_loss: 5.7925  avg_val_loss: 5.4017 F1: 0.0017  Acc: 0.0401 LR: 0.0099time: 944s
2022-04-07 22:38:37,459 DEBUG   Epoch 1 - Save Best Accuracy: 0.040067 Model
2022-04-07 22:38:38,100 DEBUG   Epoch 1 - Save Best Loss: 5.4017 Model
1725it [03:50,  7.65it/s]

In [ ]:
torch.save(model.state_dict(), f'iNat18-ViT_base_patch32_224-TaxFormer-100E.pth')

### Testing

In [22]:
model.load_state_dict(torch.load('iNat18-ViT_base_patch32_224-i21k-100E-2.pth'))

<All keys matched successfully>

In [23]:
from sklearn.metrics import f1_score, accuracy_score
import time
import tqdm

timecek = time.time()

model.to(device)
model.eval()

preds = np.zeros((len(test_dataset)))
preds_raw = []
criterion = nn.CrossEntropyLoss()
all_labels = np.zeros((len(test_dataset)))
wrong_paths = []

for i, (images, labels) in tqdm.tqdm(enumerate(test_loader), total=len(test_loader)):

    images = images.to(device)
    labels = labels.to(device)

    with torch.no_grad():
        y_preds = model(images)
    preds[i * BATCH_SIZE: (i+1) * BATCH_SIZE] = y_preds.argmax(1).to('cpu').numpy()
    all_labels[i * BATCH_SIZE: (i+1) * BATCH_SIZE] = labels.to('cpu').numpy()

    preds_raw.extend(y_preds.to('cpu').numpy())

accuracy = accuracy_score(test_metadata['class_id'], preds)

print('F1:', score, 'Vanila Accuracy:', accuracy)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 274/274 [00:21<00:00, 12.67it/s]

F1: 0.5317145992427621 Vanila Accuracy: 0.5723917266598103


In [13]:
model.load_state_dict(torch.load('iNat18-ViT_base_patch32_224-i21k_best_accuracy-2.pth'))

<All keys matched successfully>

In [17]:
from sklearn.metrics import f1_score, accuracy_score
import time
import tqdm

timecek = time.time()

model.to(device)
model.eval()

preds = np.zeros((len(test_dataset)))
preds_raw = []
criterion = nn.CrossEntropyLoss()
all_labels = np.zeros((len(test_dataset)))
wrong_paths = []

for i, (images, labels) in tqdm.tqdm(enumerate(test_loader), total=len(test_loader)):

    images = images.to(device)
    labels = labels.to(device)

    with torch.no_grad():
        y_preds = model(images)
    preds[i * BATCH_SIZE: (i+1) * BATCH_SIZE] = y_preds.argmax(1).to('cpu').numpy()
    all_labels[i * BATCH_SIZE: (i+1) * BATCH_SIZE] = labels.to('cpu').numpy()

    preds_raw.extend(y_preds.to('cpu').numpy())

print(time.time() - timecek)

accuracy = accuracy_score(test_metadata['class_id'], preds)

print('F1:', score, 'Vanila Accuracy:', accuracy)

F1: 0.5317145992427621 Vanila Accuracy: 0.5707919094960576


In [18]:
from scipy.special import softmax

In [19]:
test_metadata['predictions'] = preds
test_metadata['predictions_raw'] = preds_raw

test_metadata['predictions_softmax'] = [softmax(row) for row in test_metadata['predictions_raw']]
test_metadata['predictions_max'] = [np.max(softmax(row)) for row in test_metadata['predictions_raw']]
test_metadata['predictions_max_logits'] = [np.max(row) for row in test_metadata['predictions_raw']]
test_metadata['predictions'] = test_metadata['predictions'].astype('int32')
test_metadata

,Unnamed: 0,license,file_name,rights_holder,height,width,id,image_id,category_id,class_id,image_path,predictions,predictions_raw,predictions_softmax,predictions_max,predictions_max_logits
0,0,3,train_val2018/Plantae/7291/9f8ecb39e5f767e39e9...,Ashley M Bradford,574,800,437514,437514,7291,2070,/Data-10T/iNaturalist2018/train_val2018/Planta...,2070,"[1.1651709, -2.2287421, 1.8892187, -2.7349014,...","[1.6985521e-10, 5.703233e-12, 3.5037184e-10, 3...",0.998009,23.659252
1,1,3,train_val2018/Plantae/7228/cfb9ca114758b134ae2...,ellen hildebrandt,512,800,437516,437516,7228,2007,/Data-10T/iNaturalist2018/train_val2018/Planta...,2007,"[4.119142, 1.6230068, 5.297834, 1.2901665, 2.3...","[2.4639728e-08, 2.030383e-09, 8.0082074e-08, 1...",0.636077,21.185612
2,2,3,train_val2018/Plantae/6056/4af0d3da8aa2a7d1d1d...,aforrestel,800,597,437518,437518,6056,835,/Data-10T/iNaturalist2018/train_val2018/Planta...,866,"[1.1656909, -0.9130016, 1.323849, -0.7991873, ...","[2.1845125e-08, 2.7326845e-09, 2.5588301e-08, ...",0.599276,18.292946
3,3,3,train_val2018/Plantae/6127/efde4966d69768758f7...,Colin Murray,800,600,437520,437520,6127,906,/Data-10T/iNaturalist2018/train_val2018/Planta...,906,"[-3.2462316, -1.2571318, -2.5196042, 2.7179542...","[4.9283265e-11, 3.6020886e-10, 1.0192241e-10, ...",0.993246,20.480429
4,4,3,train_val2018/Plantae/8087/a9358012451f8f0849d...,Laura Clark,600,800,437521,437521,8087,2866,/Data-10T/iNaturalist2018/train_val2018/Planta...,2866,"[-0.9858758, 2.2382329, 4.4613104, 1.6204271, ...","[2.3398056e-10, 5.880209e-09, 5.4307925e-08, 3...",0.944553,21.132864
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8746,8746,3,train_val2018/Plantae/7839/8b943eda35c484f3713...,Jon Sullivan,535,800,461930,461930,7839,2618,/Data-10T/iNaturalist2018/train_val2018/Planta...,2617,"[2.116208, 9.033651, 5.282434, 4.7096963, 2.25...","[4.0749984e-07, 0.00041146687, 9.665005e-06, 5...",0.533276,16.200718
8747,8747,3,train_val2018/Plantae/5823/02070afb40961103d32...,parkecology,800,600,461931,461931,5823,602,/Data-10T/iNaturalist2018/train_val2018/Planta...,1027,"[-1.593773, 7.7523327, 3.469642, 6.1941366, -1...","[1.5938522e-08, 0.00018256188, 2.5203512e-06, ...",0.765055,16.092947
8748,8748,3,train_val2018/Plantae/5934/916d0b2328e9fc264ef...,tiyumq,600,800,461935,461935,5934,713,/Data-10T/iNaturalist2018/train_val2018/Planta...,670,"[-1.7736619, 8.416097, 4.3295345, 4.917265, -0...","[3.259108e-08, 0.0008678705, 1.4577517e-05, 2....",0.255860,14.102442
8749,8749,3,train_val2018/Plantae/5920/b2438ef8da565bc1b8e...,Andy Bridges,800,600,461937,461937,5920,699,/Data-10T/iNaturalist2018/train_val2018/Planta...,940,"[-0.12051439, -2.4259937, -1.0272853, 0.441875...","[4.701857e-09, 4.6882703e-10, 1.8987332e-09, 8...",0.478670,18.318050


In [21]:
wrong_classification = test_metadata[test_metadata['class_id'] != test_metadata['predictions']].reset_index()
wrong_classification = wrong_classification.reset_index()
wrong_classification

,level_0,index,Unnamed: 0,license,file_name,rights_holder,height,width,id,image_id,category_id,class_id,image_path,predictions,predictions_raw,predictions_softmax,predictions_max,predictions_max_logits,mode_prediction
0,0,2,2,3,train_val2018/Plantae/6056/4af0d3da8aa2a7d1d1d...,aforrestel,800,597,437518,437518,6056,835,/Data-10T/iNaturalist2018/train_val2018/Planta...,866,"[1.1656909, -0.9130016, 1.323849, -0.7991873, ...","[2.1845125e-08, 2.7326845e-09, 2.5588301e-08, ...",0.599276,18.292946,866
1,1,8,8,3,train_val2018/Plantae/8023/37b8118ce86472605bb...,Muriel Bendel,800,600,437533,437533,8023,2802,/Data-10T/iNaturalist2018/train_val2018/Planta...,2797,"[2.144752, 1.8515866, 3.0316086, 3.8272204, 0....","[5.968052e-08, 4.4515634e-08, 1.4487362e-07, 3...",0.383132,17.819637,2797
2,2,9,9,1,train_val2018/Plantae/7470/0e7b7fb2a3201b17505...,Gary Griffith,800,600,437536,437536,7470,2249,/Data-10T/iNaturalist2018/train_val2018/Planta...,2248,"[3.0566068, 2.9556508, 0.3731932, 1.9103473, 0...","[1.1446525e-08, 1.0347348e-08, 7.821362e-10, 3...",0.986700,21.328796,2248
3,3,11,11,3,train_val2018/Plantae/5578/5e83631b11667e7e8d6...,Liana May,800,600,437539,437539,5578,357,/Data-10T/iNaturalist2018/train_val2018/Planta...,484,"[-0.7459626, 6.7633533, 5.3928595, 6.7342777, ...","[1.1829361e-07, 0.00021588178, 5.4829972e-05, ...",0.596296,14.687115,484
4,4,13,13,3,train_val2018/Plantae/7961/316b90ecdcbc1353bc0...,Erika Mitchell,800,534,437544,437544,7961,2740,/Data-10T/iNaturalist2018/train_val2018/Planta...,2746,"[2.311655, -1.2216866, 1.2084817, 2.5623944, 0...","[1.8033896e-08, 5.267184e-10, 5.9839382e-09, 2...",0.988844,20.131449,2746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3751,3751,8746,8746,3,train_val2018/Plantae/7839/8b943eda35c484f3713...,Jon Sullivan,535,800,461930,461930,7839,2618,/Data-10T/iNaturalist2018/train_val2018/Planta...,2617,"[2.116208, 9.033651, 5.282434, 4.7096963, 2.25...","[4.0749984e-07, 0.00041146687, 9.665005e-06, 5...",0.533276,16.200718,2617
3752,3752,8747,8747,3,train_val2018/Plantae/5823/02070afb40961103d32...,parkecology,800,600,461931,461931,5823,602,/Data-10T/iNaturalist2018/train_val2018/Planta...,1027,"[-1.593773, 7.7523327, 3.469642, 6.1941366, -1...","[1.5938522e-08, 0.00018256188, 2.5203512e-06, ...",0.765055,16.092947,1027
3753,3753,8748,8748,3,train_val2018/Plantae/5934/916d0b2328e9fc264ef...,tiyumq,600,800,461935,461935,5934,713,/Data-10T/iNaturalist2018/train_val2018/Planta...,670,"[-1.7736619, 8.416097, 4.3295345, 4.917265, -0...","[3.259108e-08, 0.0008678705, 1.4577517e-05, 2....",0.255860,14.102442,670
3754,3754,8749,8749,3,train_val2018/Plantae/5920/b2438ef8da565bc1b8e...,Andy Bridges,800,600,461937,461937,5920,699,/Data-10T/iNaturalist2018/train_val2018/Planta...,940,"[-0.12051439, -2.4259937, -1.0272853, 0.441875...","[4.701857e-09, 4.6882703e-10, 1.8987332e-09, 8...",0.478670,18.318050,940
